------------
> ❗❗❗⚠️  💸   DONT USE GPU❗❗❗   
> **🧠 🤑   USE ONLY CPU (max GB used: RAM 3 - DISC 30)**
------------

## Introducción

En este proyecto, se implementará un flujo de trabajo integral para la evaluación y selección de modelos de Machine Learning utilizando las herramientas `MLflow` y `pyngrok`. En primer lugar, se instalarán las librerías necesarias y se configurará el entorno de trabajo, aprovechando Google Colab y Google Drive para la gestión y almacenamiento de datos. Los datasets de entrenamiento y prueba, junto con los modelos preentrenados y los resultados de los pipelines, se cargarán y verificarán para asegurar que todos los recursos necesarios estén disponibles para el análisis.

A continuación, se utilizará `ngrok` para establecer un túnel que permitirá acceder a la interfaz de usuario de `MLflow` desde una URL pública. Esto facilitará la visualización y gestión remota de los experimentos de `MLflow`, proporcionando una forma eficiente de monitorear y analizar los resultados de los modelos de Machine Learning. Se iniciará el servidor de `MLflow` y se expondrá a través de `ngrok`, permitiendo una integración fluida y accesible.

Finalmente, se llevará a cabo un proceso sistemático para identificar el mejor modelo entre los evaluados. La función `best_model(results_pipeline)` recorrerá los resultados de los modelos almacenados, registrando métricas y parámetros en `MLflow`, y seleccionando el modelo con el mejor rendimiento basado en la precisión media y la desviación estándar. Este enfoque garantizará que los resultados sean documentados de manera organizada y accesible, facilitando la selección del modelo más adecuado para futuras implementaciones.

In [ ]:
!pip install -q mlflow pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import joblib
import subprocess
import time
import mlflow
import sklearn
import sklearn.linear_model as lm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from google.colab import drive
from google.colab import userdata
from pyngrok import ngrok

In [ ]:
# URL de la carpeta compartida para el modelo y el diccionario
folder_url_modelo = 'https://drive.google.com/drive/folders/1wQv6lTixINg17x9E2JNhWoDO1fi7l4Xx?usp=sharing'

In [ ]:
# Extraemos el ID de la carpeta del modelo y el diccionario
folder_id_modelo = folder_url_modelo.split('/')[-1].split('?')[0]

In [ ]:
# Listamos los archivos en la carpeta usando gdown
!gdown --folder {folder_id_modelo}

Retrieving folder contents
Processing file 1w7blx-lYxGVdkmyL67LklXXs8tKj00Vt dict_model_results.pkl
Processing file 1yuWQajrhukyKLHDkt0BeWajoNPF_djUY model_random_forest.pkl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1w7blx-lYxGVdkmyL67LklXXs8tKj00Vt
To: /content/FraudDetection_Pipeline/dict_model_results.pkl
100% 2.62k/2.62k [00:00<00:00, 12.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yuWQajrhukyKLHDkt0BeWajoNPF_djUY
To: /content/FraudDetection_Pipeline/model_random_forest.pkl
100% 719k/719k [00:00<00:00, 13.8MB/s]
Download completed


In [ ]:
dict_model_results = '/content/FraudDetection_Pipeline/dict_model_results.pkl'
model_random_forest = '/content/FraudDetection_Pipeline/model_random_forest.pkl'

In [ ]:
model_randomforest = joblib.load(model_random_forest)
results_pipeline = joblib.load(dict_model_results)

In [ ]:
# Verificar que el modelo y los resultados se ha cargado correctamente
print(model_randomforest)
print(results_pipeline)

RandomForestClassifier(n_estimators=25)
{'names': ['Random Forest', 'LASSO', 'KNN', 'Decision Tree', 'SVM', 'Gradient Boosting'], 'results': [array([0.99700832, 0.99648803, 0.99596774, 0.99687785, 0.99609731]), array([0.98907388, 0.98868366, 0.98725286, 0.98816183, 0.98660075]), array([0.98478148, 0.98426119, 0.9866025 , 0.98360869, 0.98503968]), array([0.99544745, 0.99544745, 0.99362643, 0.99440614, 0.99453623]), array([0.99063476, 0.99011446, 0.99037461, 0.98907246, 0.98868219]), array([0.99583767, 0.9951873 , 0.99440687, 0.99544686, 0.9949265 ])], 'models': [('Random Forest', RandomForestClassifier(random_state=42)), ('LASSO', LogisticRegression(random_state=42)), ('KNN', KNeighborsClassifier()), ('Decision Tree', DecisionTreeClassifier(random_state=42)), ('SVM', SVC(random_state=42)), ('Gradient Boosting', GradientBoostingClassifier(random_state=42))]}


In [ ]:
# Empezamos a probar mlfflow y ngrok
ngrok.kill()

NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print('El tracking UI:', ngrok_tunnel.public_url)

El tracking UI: https://3a52-35-186-188-202.ngrok-free.app


In [ ]:
# Corremos todos los modelos utilizando el modelo y resultados guardados
def best_model(results_pipeline):

  # setemos en 0 los mejores resultados
  best_mean_result = 0
  best_std_result = 0

  mlflow_ui_process = subprocess.Popen(['mlflow', 'ui', '--port', '5000'])

  time.sleep(5)

  mlflow.set_experiment("Script Eleccion Modelo")

  names = results_pipeline['names']
  results = results_pipeline['results']
  models = results_pipeline['models']

  for i in range(len(results)):
    with mlflow.start_run(run_name=results[i][0]) as run:
      cv_results = results[i]
      name = names[i]
      model = models[i]
      print(model)
      mlflow.log_metric('m1', np.mean(cv_results))
      mlflow.log_param('model', name)
      mlflow.log_param('accuracy', cv_results)
      mlflow.sklearn.log_model(model, 'clf_model')

      names.append(name)
      print(name + ": mean(accuracy)=" + str(round(np.mean(cv_results), 3)) + ", std(accuracy)=" + str(round(np.std(cv_results), 3)))

      if (best_mean_result < np.mean(cv_results)) or \
        ((best_mean_result == np.mean(cv_results)) and (best_std_result > np.std(cv_results))):
        best_mean_result = np.mean(cv_results)
        best_std_result = np.std(cv_results)
        best_model_name = name
        best_model = model

  return best_model, best_model_name

In [ ]:
# probamos la generación de resultados y login en mlflow

best_model(results_pipeline)

2024/07/18 15:30:21 INFO mlflow.tracking.fluent: Experiment with name 'Script Eleccion Modelo' does not exist. Creating a new experiment.
2024/07/18 15:30:21 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


('Random Forest', RandomForestClassifier(random_state=42))


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/07/18 15:30:25 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Random Forest: mean(accuracy)=0.996, std(accuracy)=0.0
('LASSO', LogisticRegression(random_state=42))


2024/07/18 15:30:27 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


...

2024/07/18 15:30:31 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Decision Tree: mean(accuracy)=0.995, std(accuracy)=0.001
('SVM', SVC(random_state=42))


2024/07/18 15:30:33 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


SVM: mean(accuracy)=0.99, std(accuracy)=0.001
('Gradient Boosting', GradientBoostingClassifier(random_state=42))
Gradient Boosting: mean(accuracy)=0.995, std(accuracy)=0.0


(('Random Forest', RandomForestClassifier(random_state=42)), 'Random Forest')